In [1]:
import pickle
import os
import numpy as np
from tqdm import tqdm 
from scipy.io import wavfile
from python_speech_features import mfcc
from keras.models import load_model
import pandas as pd
from sklearn.metrics import accuracy_score


Using TensorFlow backend.


In [2]:
def build_predictions(audio_dir):
    y_true = []
    y_pred = []
    fn_prob = {}
    
    print('Extracting features from audio')
    for fn in tqdm(os.listdir(audio_dir)):
        rate, wav = wavfile.read(os.path.join(audio_dir,fn))
        label = fn2class[fn]
        c = classes.index(label)
        y_prob = []
        
        for i in range(0, wav.shape[0]-config.step, config.step):
            sample = wav[i:i+config.step]
            x = mfcc(sample,rate,numcep=config.nfft)
            x = (x - config.min)/(config.max - config.min)
            print("1: ",x)
            print("2: ",x.shape)
            if config.mode == 'conv':
                x = x.reshape(1,x.shape[0],x.shape[1],1)
            elif config.mode == 'time':
                x = np.expand_dims(x, axis =0)
            print("3: ",x)
            print("4: ",x.shape)
            y_hat = model.predict(x)
            y_prob.append(y_hat)
            y_pred.append(np.argmax(y_hat))
            y_true.append(c)
            
        fn_prob[fn] = np.mean(y_prob, axis = 0).flatten()
        
        return y_true, y_pred, fn_prob        
        
df = pd.read_csv('instruments.csv')
classes = list(np.unique(df.label))
fn2class = dict(zip(df.fname,df.label))
p_path = os.path.join('pickles','conv.p')

with open(p_path,'rb') as handle:
    config = pickle.load(handle)
    
model = load_model(config.model_path)

y_true, y_pred, fn_prob = build_predictions('clean')
acc_score = accuracy_score(y_true=y_true, y_pred=y_pred)

y_probs = []
for i,row in df.iterrows():
    y_prob = fn_prob[row.fname]
    y_probs.append(y_prob)
    for c, p in zip(classes, y_prob):
        df.at[i,c] = p
        
y_pred = [classes[np.argmax(y)] for y in y_probs]
df['y_pred'] = y_pred

df.to_csv('prediction.csv', index = False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


  0%|          | 0/300 [00:00<?, ?it/s]

Extracting features from audio
1:  [[0.43420565 0.32333558 0.33790023 0.47740222 0.52595075 0.56914479
  0.38720302 0.44385069 0.46406029 0.43607325 0.44051126 0.59854384
  0.65468489 0.69269866 0.61702269 0.48956341 0.4582976  0.55665777
  0.55445548 0.47831103 0.47282085 0.48864372 0.50076145 0.49979162
  0.505381   0.5135634 ]
 [0.43478439 0.33585775 0.36246559 0.5227536  0.57539743 0.59352924
  0.3988229  0.44821595 0.47498781 0.47270032 0.45073271 0.56729464
  0.59019357 0.62071526 0.63560419 0.5241704  0.44137457 0.52041344
  0.54067716 0.49030345 0.48578573 0.49778308 0.50226306 0.50079117
  0.50369976 0.50336877]
 [0.4353019  0.32549048 0.33289057 0.49263992 0.5284956  0.53225196
  0.35460849 0.4201758  0.48100713 0.47968467 0.49364128 0.61122036
  0.61654162 0.62869669 0.61131791 0.53460401 0.46767114 0.54524197
  0.54677012 0.49800674 0.50316494 0.50086952 0.50349205 0.5001807
  0.5035508  0.49972787]
 [0.43726595 0.35489823 0.35262923 0.51448083 0.53951784 0.54818212
  0.352

ValueError: Error when checking input: expected conv2d_1_input to have shape (9, 13, 1) but got array with shape (9, 26, 1)